In [ ]:
!pip install datasets
!pip install accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [ ]:
!gdown --id 11pIbZPWaadmk8gzaZG6tpRd1rGoPUa8Q

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=11pIbZPWaadmk8gzaZG6tpRd1rGoPUa8Q
From (redirected): https://drive.google.com/uc?id=11pIbZPWaadmk8gzaZG6tpRd1rGoPUa8Q&confirm=t&uuid=656d1553-b13a-41cd-b6ca-2f42190644f5
To: /content/Dataset.zip
100% 60.2M/60.2M [00:01<00:00, 52.1MB/s]


In [ ]:
!unzip Dataset.zip -d Dataset

Archive:  Dataset.zip
  inflating: Dataset/Dictionary.txt  
  inflating: Dataset/Meta-data/Data.xlsx  
  inflating: Dataset/ReadMe.txt      
  inflating: Dataset/WOS11967/desktop.ini  
  inflating: Dataset/WOS11967/X.txt  
  inflating: Dataset/WOS11967/Y.txt  
  inflating: Dataset/WOS11967/YL1.txt  
  inflating: Dataset/WOS11967/YL2.txt  
  inflating: Dataset/WOS46985/desktop.ini  
  inflating: Dataset/WOS46985/X.txt  
  inflating: Dataset/WOS46985/Y.txt  
  inflating: Dataset/WOS46985/YL1.txt  
  inflating: Dataset/WOS46985/YL2.txt  
  inflating: Dataset/WOS5736/desktop.ini  
  inflating: Dataset/WOS5736/X.txt   
  inflating: Dataset/WOS5736/Y.txt   
  inflating: Dataset/WOS5736/YL1.txt  
  inflating: Dataset/WOS5736/YL2.txt  


In [ ]:
# Download and extract data
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import os
import os
import re
def text_cleaner(text):
    """
    cleaning spaces, html tags, etc
    parameters: (string) text input to clean
    return: (string) clean_text
    """
    text = text.replace(".", "")
    text = text.replace("[", " ")
    text = text.replace(",", " ")
    text = text.replace("]", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("\"", "")
    text = text.replace("-", "")
    text = text.replace("=", "")
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
        for (k, v) in rule.items():
            regex = re.compile(k)
            text = regex.sub(v, text)
        text = text.rstrip()
        text = text.strip()
    clean_text = text.lower()
    return clean_text
path_WOS = "Dataset"  # Update with the actual path
fname = os.path.join(path_WOS, "WOS5736/X.txt")
fnamek = os.path.join(path_WOS, "WOS5736/YL1.txt")
fnameL2 = os.path.join(path_WOS, "WOS5736/YL2.txt")

# Read and clean content
with open(fname) as f:
    content = f.readlines()
    content = [text_cleaner(x) for x in content]

with open(fnamek) as fk:
    contentk = fk.readlines()
contentk = [x.strip() for x in contentk]

with open(fnameL2) as fk:
    contentL2 = fk.readlines()
    contentL2 = [x.strip() for x in contentL2]

Label = np.matrix(contentk, dtype=int)
Label = np.transpose(Label)
number_of_classes_L1 = np.max(Label) + 1  # number of classes in Level 1

Label_L2 = np.matrix(contentL2, dtype=int)
Label_L2 = np.transpose(Label_L2)
np.random.seed(7)
Label = np.column_stack((Label, Label_L2))
number_of_classes_L2 = np.zeros(number_of_classes_L1, dtype=int)

X_train, X_test, y_train, y_test = train_test_split(content, Label, test_size=0.2, random_state=0)

L2_Train = [[] for _ in range(number_of_classes_L1)]
L2_Test = [[] for _ in range(number_of_classes_L1)]
content_L2_Train = [[] for _ in range(number_of_classes_L1)]
content_L2_Test = [[] for _ in range(number_of_classes_L1)]

for i in range(len(X_train)):
    L2_Train[y_train[i, 0]].append(y_train[i, 1])
    number_of_classes_L2[y_train[i, 0]] = max(number_of_classes_L2[y_train[i, 0]], (y_train[i, 1] + 1))
    content_L2_Train[y_train[i, 0]].append(X_train[i])

for i in range(len(X_test)):
    L2_Test[y_test[i, 0]].append(y_test[i, 1])
    content_L2_Test[y_test[i, 0]].append(X_test[i])

for i in range(number_of_classes_L1):
    L2_Train[i] = np.array(L2_Train[i])
    L2_Test[i] = np.array(L2_Test[i])
    content_L2_Train[i] = np.array(content_L2_Train[i])
    content_L2_Test[i] = np.array(content_L2_Test[i])

In [ ]:

from datasets import Dataset, load_metric
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
def preprocess_data(tokenizer, texts, labels, max_length=512):
    encoded_inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    dataset = Dataset.from_dict({
        'input_ids': encoded_inputs['input_ids'],
        'attention_mask': encoded_inputs['attention_mask'],
        'labels': torch.tensor(labels)
    })
    return dataset

def compute_metrics(pred):
    metric = load_metric("accuracy")
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def train_bert_classifier(X_train, y_train, X_test, y_test, number_of_classes):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    train_dataset = preprocess_data(tokenizer, X_train, y_train)
    test_dataset = preprocess_data(tokenizer, X_test, y_test)

    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=number_of_classes)

    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=10,              # total number of training epochs
        per_device_train_batch_size=16,   # batch size for training
        per_device_eval_batch_size=8,    # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=test_dataset,           # evaluation dataset
        compute_metrics=compute_metrics,     # evaluation metrics
    )

    trainer.train()
    trainer.evaluate()

    # Evaluate on test data
    predictions = trainer.predict(test_dataset)
    y_pred = np.argmax(predictions.predictions, axis=-1)
    print(classification_report(y_test, y_pred))
    return y_test, y_pred

In [ ]:
y_train = [x[0] for x in y_train]
y_test = [x[0] for x in y_test]

In [ ]:
train_bert_classifier(list(X_train),y_train, list(X_test),y_test, number_of_classes = number_of_classes_L1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.111300
20,1.070300
30,0.991600
40,0.940400
50,0.916500
60,0.832000
70,0.762300
80,0.639900
90,0.598100
100,0.461400


<ipython-input-46-6aae131c6c64>:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       257
           1       0.97      0.98      0.98       307
           2       1.00      1.00      1.00       584

    accuracy                           0.99      1148
   macro avg       0.99      0.99      0.99      1148
weighted avg       0.99      0.99      0.99      1148



([2,
  2,
  2,
  0,
  1,
  0,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  0,
  1,
  2,
  2,
  1,
  1,
  1,
  2,
  2,
  0,
  2,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  2,
  0,
  0,
  0,
  1,
  2,
  1,
  1,
  2,
  0,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  1,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  2,
  0,
  1,
  1,
  1,
  0,
  0,
  2,
  1,
  1,
  0,
  1,
  2,
  2,
  2,
  2,
  2,
  1,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  0,
  0,
  2,
  0,
  2,
  0,
  1,
  0,
  2,
  0,
  2,
  0,
  1,
  2,
  0,
  2,
  0,
  1,
  2,
  2,
  0,
  2,
  0,
  2,
  2,
  1,
  1,
  2,
  1,
  2,
  0,
  0,
  2,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  1,
  1,
  2,
  1,
  0,
  2,
  0,
  2,
  0,
  2,
  1,
  1,
  1,
  2,
  2,
  0,
  1,
  1,
  2,
  2,
  2,
  2,
  1,
  2,
  0,
  2,
  2,
  2,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  2,
  1,
  0,
  2,


In [ ]:
clf_L2 = []
y_train = np.array(y_train)
y_test = np.array(y_test)
label =[]
predictions = []
for i in range(number_of_classes_L1):
    number_of_classes_L2 = max(L2_Train[i]) +1
    y_test, y_pred = train_bert_classifier(list(content_L2_Train[i]), L2_Train[i], list(content_L2_Test[i]), L2_Test[i], number_of_classes = number_of_classes_L2)
    label.extend(list(y_test))
    predictions.extend(list(y_pred))

print(classification_report(label, predictions))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.082100
20,1.093200
30,1.051200
40,1.029700
50,0.970800
60,0.913000
70,0.858600
80,0.778300
90,0.717500
100,0.616800


              precision    recall  f1-score   support

           0       0.99      0.97      0.98        87
           1       0.96      0.94      0.95        79
           2       0.95      0.99      0.97        91

    accuracy                           0.96       257
   macro avg       0.97      0.96      0.96       257
weighted avg       0.97      0.96      0.96       257



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.418400
20,1.405900
30,1.401400
40,1.334100
50,1.280400
60,1.310600
70,1.227400
80,1.136000
90,1.148300
100,0.996200


              precision    recall  f1-score   support

           0       0.95      0.88      0.91        84
           1       0.95      0.93      0.94        80
           2       0.84      0.89      0.87        66
           3       0.90      0.95      0.92        77

    accuracy                           0.91       307
   macro avg       0.91      0.91      0.91       307
weighted avg       0.91      0.91      0.91       307



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.438400
20,1.407500
30,1.423800
40,1.391100
50,1.344600
60,1.352700
70,1.332000
80,1.310300
90,1.278400
100,1.204500


              precision    recall  f1-score   support

           0       0.93      0.88      0.90       165
           1       0.93      0.91      0.92       125
           2       0.88      0.95      0.91       154
           3       0.99      0.99      0.99       140

    accuracy                           0.93       584
   macro avg       0.93      0.93      0.93       584
weighted avg       0.93      0.93      0.93       584

              precision    recall  f1-score   support

           0       0.95      0.90      0.93       336
           1       0.94      0.92      0.93       284
           2       0.89      0.95      0.92       311
           3       0.96      0.97      0.97       217

    accuracy                           0.93      1148
   macro avg       0.94      0.94      0.94      1148
weighted avg       0.93      0.93      0.93      1148



In [ ]:
number_of_classes_L1

3

In [ ]:
len(label)

1148